In [1]:
%load_ext pycodestyle_magic

In [2]:
%flake8_on

In [3]:
from itertools import count
from functools import reduce
import numpy as np

In [4]:
testdata = """939
7,13,x,x,59,x,31,19""".splitlines()

with open('input', 'r') as inp:
    inputdata = [line.strip() for line in inp.readlines()]

In [5]:
def process_data(data):
    return (int(data[0]),
            [(int(id), o) for o, id in
                enumerate(data[1].split(','))
                if id != 'x'])

In [6]:
def waiting_times(earliest, ids):
    return [(0, id) if (earliest % id == 0)
            else (id - (earliest % id), id)
            for id, o in ids]

In [7]:
def offset_errors(time, ids):
    return [(0, id) if (time % id) == o
            else (((time % id) + o) % id, id)
            for id, o in ids]

In [8]:
def print_schedule(earliest, ids):
    wtimes_ids = waiting_times(earliest, ids)
    wtimes = [wtime for wtime, id in wtimes_ids]
    # os = [o for id, o in ids]
    ids = [id for id, o in ids]
    # print(f'waiting times: {wtimes}')
    # print(f'offsets: {os}')
    # print(f'bus ids: {ids}')
    print(f'time    {"  ".join(["bus " + str(id) for id in ids])}')
    for t in range(0, max(wtimes) + 1):
        print('{}    {}'.format(
            t + earliest,
            "     ".join([
                " D " if (t + earliest) % id == 0
                else " . "
                for id in ids
            ])
        ))

For $B_a$ and $B_b$ as two bus ids in this context (also their period), where $B_a$ is assumed to have $o_a$ offset (or "phase") and $B_b$ offset $o_b$, let $d_{a,b} = gcd(B_a, B_b)$

By [Euclid's lemma](https://en.wikipedia.org/wiki/Euclid%27s_lemma), there exist integers $N_a, N_b$ such that $N_a ⋅ B_a - N_b ⋅ B_b = c$

We are looking for $m, n$ so that $m⋅B_a − o_a = n⋅B_b − o_b => m⋅B_a − n⋅B_b = o_a − o_b$ $(1)$. Using Euclid's extended algorithm, we can find $s, t$ so that $s⋅B_a + t⋅B_b = gcd(B_a, B_b)$ $(2)$. Assuming $mod(o_a - o_b, g_{a,b}) = 0$ (there exists a solution), let $z = \frac{o_a - o_b}{g_{a,b}}$, multiply $(2)$ with it to get $(1)$ as $m = z⋅s$ and $n = −z⋅t$

In [9]:
def extended_gcd(a, b):
    """Extended Greatest Common Divisor Algorithm

    Returns:
        gcd: The greatest common divisor of a and b.
        s, t: Coefficients such that s*a + t*b = gcd

    Reference:
        https://en.wikipedia.org/wiki/Extended_Euclidean_algorithm#Pseudocode
    """
    old_r, r = a, b
    old_s, s = 1, 0
    old_t, t = 0, 1
    while r:
        quotient, remainder = divmod(old_r, r)
        old_r, r = r, remainder
        old_s, s = s, old_s - quotient * s
        old_t, t = t, old_t - quotient * t

    return old_r, old_s, old_t

In [10]:
def combine_phased_schedules(a_period, a_phase, b_period, b_phase):
    gcd, s, t = extended_gcd(a_period, b_period)
    phase_difference = a_phase - b_phase
    pd_mult, pd_remainder = divmod(phase_difference, gcd)
    if pd_remainder:
        raise ValueError("Schedules never synchronize")
    combined_period = a_period // gcd * b_period
    combined_phase = (a_phase - s * pd_mult * a_period) % combined_period
    return combined_period, combined_phase

In [11]:
def schedule_alignment(id_a, id_b):
    a_period, a_offset = id_a
    b_period, b_offset = id_b
    """Where the arrows first align, where green starts shifted by advantage"""
    period, phase = combine_phased_schedules(a_period, a_offset % a_period,
                                             b_period, b_offset % b_period)
    print(period, -phase % period)
    return (period, -phase % period)

In [12]:
def chinese_remainder(n, a):
    sum = 0
    prod = reduce(lambda a, b: a*b, n)
    for n_i, a_i in zip(n, a):
        p = prod // n_i
        sum += a_i * mul_inv(p, n_i) * p
    return sum % prod


def mul_inv(a, b):
    b0 = b
    x0, x1 = 0, 1
    if b == 1:
        return 1
    while a > 1:
        q = a // b
        a, b = b, a % b
        x0, x1 = x1 - q * x0, x0
    if x1 < 0:
        x1 += b0
    return x1

In [13]:
earliest, ids = process_data(testdata)
print(earliest, ids)
times = waiting_times(earliest, ids)
print(f'times: {times}')
shortest = min(times)
print(f'shortest waiting time * that bus Id: {shortest[0]*shortest[1]}')

# print(schedule_alignment(ids[0], ids[1]))
# print(schedule_alignment(ids[2], ids[3]))
# print(schedule_alignment((91, 77), (1829, 645)))
# print(schedule_alignment((166439, 96292), ids[4]))

# print(reduce(lambda a, b: schedule_alignment(a, b), ids))
# print_schedule(1068781, ids)

# print(schedule_alignment(166439, 98357, 19, 7))
idx = [0, 1, 2, 3, 4]
print_schedule(3162341+1068781, np.array(ids)[idx])
2093560
earliest, ids = process_data(inputdata)
print(ids)

print(schedule_alignment(schedule_alignment(schedule_alignment(schedule_alignment((29, 0), (41, 19)), schedule_alignment((661, 29), (13, 42))), schedule_alignment(schedule_alignment((17, 43), (23, 52)), schedule_alignment((521, 60), (37, 66)))), (19, 79)))

print(offset_errors(864238811652128, ids))

print(chinese_remainder([id for id, o in ids], [o for id, o in ids]))

#print_schedule(864238811652128, ids)
# for t in count(0, max(ids)[0]):
#     #print_schedule(t, ids)
#     wtimes_ids = waiting_times(t, ids)
#     errors = [o - t for t, o in zip([wtime for wtime, id in wtimes_ids],
#                                     [o for id, o in ids])]
#     if sum([abs(e) for e in errors]) == 0:
#         print(errors)
#         print_schedule(t, ids)
#         break



23:80: E501 line too long (256 > 79 characters)
29:1: E265 block comment should start with '# '
939 [(7, 0), (13, 1), (59, 4), (31, 6), (19, 7)]
times: [(6, 7), (10, 13), (5, 59), (22, 31), (11, 19)]
shortest waiting time * that bus Id: 295
time    bus 7  bus 13  bus 59  bus 31  bus 19
4231122     D       .       .       .       . 
4231123     .       D       .       .       . 
4231124     .       .       .       .       . 
4231125     .       .       .       .       . 
4231126     .       .       D       .       . 
4231127     .       .       .       .       . 
4231128     .       .       .       D       . 
4231129     D       .       .       .       D 
[(29, 0), (41, 19), (661, 29), (13, 42), (17, 43), (23, 52), (521, 60), (37, 66), (19, 79)]
1189 145
8593 8564
10217077 2491999
391 178
19277 18175
7537307 5668540
77009245991639 59872140247540
1463175673841141 864238811652128
(1463175673841141, 864238811652128)
[(0, 29), (0, 41), (0, 661), (6, 13), (1, 17), (12, 23), (0, 521), (21, 37

In [14]:
print(864238811652128 // 19, 864238811652128 % 19, (79 + 16) % 19)

45486253244848 16 0


In [15]:
print(864238811652128 // 41, 864238811652128 % 41)

21078995406149 19


In [16]:
print(8564 // 13, 8564 % 13, (42 + 10) % 13)

658 10 0


In [17]:
2093560 // 2 # 1068781?

1046780

1:13: E261 at least two spaces before inline comment


In [18]:
earliest, ids = process_data(testdata)
print(ids)
# print_schedule(2093560, ids)

# 1068781
print(f'chinese remainder: {chinese_remainder([id for id, o in ids], [-o for id, o in ids])}')

print(1068781 % ids[3][0])

6:80: E501 line too long (94 > 79 characters)
[(7, 0), (13, 1), (59, 4), (31, 6), (19, 7)]
chinese remainder: 1068781
25


In [19]:
earliest, ids = process_data(inputdata)
print(f'chinese remainder: {chinese_remainder([id for id, o in ids], [-o for id, o in ids])}')
print(offset_errors(213890632230818, ids))

chinese remainder: 213890632230818
[(0, 29), (0, 41), (0, 661), (0, 13), (0, 17), (0, 23), (0, 521), (0, 37), (0, 19)]
2:80: E501 line too long (94 > 79 characters)
